In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

COHERE_API_KEY = os.environ["COHERE_API_KEY"]
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]
LANGCHAIN_TRACING_V2 = os.environ["LANGCHAIN_TRACING_V2"]
LANGCHAIN_ENDPOINT = os.environ["LANGCHAIN_ENDPOINT"]
LANGCHAIN_API_KEY = os.environ["LANGCHAIN_API_KEY"]
LANGCHAIN_PROJECT = os.environ["LANGCHAIN_PROJECT"]

In [7]:
import os

from langchain_cohere import CohereEmbeddings , CohereRerank , ChatCohere
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

import cohere
from qdrant_client.models import Batch
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams



In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")
Qclient = QdrantClient(url="http://localhost:6333")
cohere_client = cohere.Client()
qclient = QdrantClient(os.environ["COHERE_API_KEY"])

In [9]:
def cohereIndexing(client,collection,texts):
    client.upsert(
    collection_name=collection,
    points=Batch(
        ids=range(len(texts)),
        vectors=cohere_client.embed(
            model="embed-english-v3.0",  # New Embed v3 model
            input_type="search_document",  # Input type for indexing documents
            texts= [texts[i].page_content for i in range(len(texts))],
        ).embeddings,
        payloads = [{"Context{}".format(index): value} for index, value in enumerate([texts[i].page_content for i in range(len(texts))], start=1)],
    ),
    )

In [10]:
def cohereRetrival(collection,textList):
    cohere_client = cohere.Client(os.environ["COHERE_API_KEY"])
    client = QdrantClient()
    result = client.search(
    collection_name=collection,
    query_vector=cohere_client.embed(
        model="embed-english-v3.0",  
        input_type="search_query",  # Input type for search queries
        texts=textList,
    ).embeddings[0],
    )
    return result

In [11]:
from langchain.prompts import ChatPromptTemplate
promptTemplate = template = """You are a medical assistant that specializes in providing second opinions, diagnosing complex cases 
and suggesting treatment plans. When I describe the patient details, medical context and task, give me the appropriate treatment plan 
or second opinion based on the task given by analyzing the patient details and medical context. In your answer include how your opinion or treatment
plann is related to the patient's history.

Patient History : {patientHistory}

Medical Context : {context}

Task: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [12]:
chain = prompt | llm

In [14]:
context = cohereRetrival("MedicalPapers",[question])

In [23]:
context[0].payload['Context813']

'4 \n 2.3 It should take place face to face in a confidential environment, with access for relatives and \nallied health professionals with a minimum time of 15 -20 minutes. Patients will often be \nmonths  or even years after injury and frustrated by the delay in diagnosis. It may be the first \ntime the diagnosis is made, but equally they may already have researched the diagnosis and \nmade  a decision on their preferred treatment. This decision may change foll owing the \nconsultation.  \n \n2.4 Past medical records from hospital and general practitioner including imaging (if any) should \nbe available.  \n \n2.5 The history of the injury, subsequent and current symptoms should be documented, followed \nby a clinical examination and re view of all imaging. Instrumented laxiometry may be helpful \nbut is not considered to be universally necessary.  \n 2.6 A management plan is made after discussion of both operative & non -operative options for'

In [13]:
patientHistory = """Patient Details : Patient Information:
Name: Rajesh Kumar
Age: 45
Gender: Male
Occupation: Office manager
Medical History:
Hypertension (controlled with medication)
Previous history of lumbar strain due to heavy lifting incidents, managed conservatively
Chief Complaint:
Chronic back pain
Duration: Approximately 6 months
Symptoms:
Dull, aching pain in the lower back, occasionally radiating down the right leg
Exacerbated by prolonged sitting or standing, relieved by lying down
Severity: 7/10 on average, worsening to 8/10 during flare-ups
No associated numbness or tingling
Impact on Daily Activities:
Difficulty performing work duties due to prolonged sitting at a desk
Limited ability to participate in recreational activities, such as playing sports or gardening
Struggles with household chores, particularly activities that involve bending or lifting
Treatment History:
Nonsteroidal anti-inflammatory drugs (NSAIDs) for pain management, with minimal relief
Physical therapy sessions focusing on core strengthening exercises and flexibility, providing short-term improvement but limited long-term benefits
Occasional use of heat packs and over-the-counter topical analgesics for symptomatic relief
Current Medications:
Amlodipine 5 mg daily for hypertension
Ibuprofen 400 mg as needed for back pain
Additional Concerns:
Rajesh is concerned about the impact of his back pain on his job performance and overall quality of life.
He's interested in exploring alternative treatment options, such as acupuncture or chiropractic care.
 """
context = """ Ibuprofen is used to relieve pain from various conditions such as headache, dental pain, menstrual cramps, muscle aches, or arthritis. It is also used to reduce fever and to relieve minor aches and pain due to the common cold or flu. Ibuprofen is a nonsteroidal anti-inflammatory drug (NSAID). It works by blocking your body's production of certain natural substances that cause inflammation. This effect helps to decrease swelling, pain, or fever.If you are treating a chronic condition such as arthritis, ask your doctor about non-drug treatments and/or using other medications to treat your pain. See also Warning section.This form of ibuprofen is intended for use in children. Do not give this medication to a child younger than 2 years unless directed by the doctor.Check the ingredients on the label even if you have used the product before. The manufacturer may have changed the ingredients. Also, products with similar names may contain different ingredients meant for different purposes. Taking the wrong product could harm you.
Amlodipine is used alone or in combination with other medications to treat high blood pressure in adults and children 6 years and older. It is also used to treat certain types of angina (chest pain) and coronary artery disease (narrowing of the blood vessels that supply blood to the heart) """

question = """ I'm facing challenges in managing a case of chronic back pain in a patient. Despite thorough evaluation and treatment attempts, including physical therapy and medication, the pain persists. The patient experiences a constant dull ache localized to the lumbar region, limiting their mobility and affecting their quality of life. I'm seeking your expertise to explore alternative treatment strategies or diagnostic approaches that may offer relief. """

In [39]:
context[0]

ScoredPoint(id=812, version=0, score=0.43539104, payload={'Context813': '4 \n 2.3 It should take place face to face in a confidential environment, with access for relatives and \nallied health professionals with a minimum time of 15 -20 minutes. Patients will often be \nmonths  or even years after injury and frustrated by the delay in diagnosis. It may be the first \ntime the diagnosis is made, but equally they may already have researched the diagnosis and \nmade  a decision on their preferred treatment. This decision may change foll owing the \nconsultation.  \n \n2.4 Past medical records from hospital and general practitioner including imaging (if any) should \nbe available.  \n \n2.5 The history of the injury, subsequent and current symptoms should be documented, followed \nby a clinical examination and re view of all imaging. Instrumented laxiometry may be helpful \nbut is not considered to be universally necessary.  \n 2.6 A management plan is made after discussion of both operati

In [40]:
result = chain.invoke({"context":retrievedDocs[0].payload['Context722'] , "patientHistory":"","question":question})
print(result)

> ## ACL Reconstruction and Meniscus Repair Recovery Plan:
> 
> Based on the provided medical context from University Orthopedics, Inc., and the patient's recent ACL reconstruction and meniscus repair, here's a comprehensive approach to ensure full recovery and prevent re-injury:
> 
> **Phase I (0-2 Weeks Post-op): Focus on Protection and Inflammation Control**
> 
> *   **Swelling Management:** This is crucial in the initial phase. Utilize RICE (Rest, Ice, Compression, Elevation) to minimize swelling and pain. Your doctor may also recommend anti-inflammatory medications.
> *   **Range of Motion (ROM):** Start gentle ROM exercises as instructed by your physical therapist to prevent stiffness and achieve the target of 0-90 degrees of flexion. 
> *   **Physical Therapy:** Begin physical therapy sessions to learn proper crutch use, gait training, and initial strengthening exercises.
> *   **Bracing:** Wear a knee brace as directed to protect the graft and provide stability.
> 
> **Phase II (2-6 Weeks Post-op): Gradual Strengthening and Mobility**
> 
> *   **Increased ROM:** Continue working towards full knee extension and further flexion as tolerated.
> *   **Strengthening:** Progress strengthening exercises to target the quadriceps, hamstrings, and calf muscles. This may involve bodyweight exercises, resistance bands, and light weights.
> *   **Proprioception Exercises:** Begin exercises to improve balance and coordination, which are crucial for knee stability and preventing re-injury.
> 
> **Phase III (6-12 Weeks Post-op): Advanced Strengthening and Functional Training**
> 
> *   **Advanced Strengthening:** Increase the intensity and complexity of strengthening exercises, incorporating more functional movements that mimic daily activities and sports-specific actions.
> *   **Plyometrics:** Introduce plyometric exercises (jumping, hopping) gradually to improve power and explosiveness.
> *   **Sport-Specific Training:** If the patient is an athlete, start incorporating sport-specific drills and exercises to prepare for a return to play.
> 
> **Long-Term Prevention of ACL Re-tear:**
> 
> *   **Continued Strengthening:** Maintain a regular exercise routine focusing on leg strength, particularly the hamstrings and quadriceps, to ensure good muscle balance and joint stability.
> *   **Neuromuscular Training:** Continue proprioception and agility exercises to improve coordination, reaction time, and dynamic stability, which are essential for preventing ACL injuries.
> *   **Proper Technique:**  Ensure the patient uses proper techniques during sports and activities, especially when pivoting, cutting, and landing.
> *   **Warm-up and Cool-down:** Always emphasize proper warm-up before activity and cool-down afterward to prepare the muscles and prevent injuries.
> *   **Bracing:** Depending on the sport and individual risk factors, consider using a brace during high-risk activities for added protection.
> 
> **Connecting with Patient History:**
> 
> Understanding the patient's history, including the mechanism of injury, activity level, and any previous injuries, is crucial in tailoring the rehabilitation program. This information helps determine the pace of progression, the specific exercises chosen, and the long-term strategies for preventing re-injury. 
> 
> **Collaboration is Key:**
> 
> It is crucial for the patient to work closely with the physical therapist and surgeon throughout the recovery process. Regular follow-up appointments will ensure proper progression and address any concerns that may arise. This collaborative approach optimizes the chances of a full recovery and a safe return to desired activities. 


In [30]:
def evaluate_with_llm(model, prompt, generated_text):
    evaluations = {}

    # Template for creating evaluation queries
    def create_evaluation_query(template, **kwargs):
        query = ChatPromptTemplate.from_template(template)
        chain = query | model
        return float(chain.invoke(kwargs).content)

    # Evaluate Relevance
    relevance_template = "Given the context provided by the following prompt: '{prompt}', please evaluate on a scale from 0 to 1, where 1 is highly relevant and 0 is not relevant at all, how relevant is this generated response: '{generated_text}'? Provide a numerical score only."
    evaluations['relevance'] = create_evaluation_query(relevance_template, prompt=prompt, generated_text=generated_text)

    # Evaluate Clarity
    clarity_template = "How clear and easily understandable is this text: '{generated_text}'? Rate its clarity on a scale from 0 to 1, where 1 indicates that the text is very clear and 0 indicates that the text is very unclear. Provide a numerical score only."
    evaluations['clarity'] = create_evaluation_query(clarity_template, prompt=prompt, generated_text=generated_text)

    # Evaluate Coherence
    coherence_template = "On a scale from 0 to 1, with 1 being highly coherent and 0 being not coherent at all, how well do the ideas in this generated text: '{generated_text}' flow together? Consider if the text makes logical sense as a whole. Provide a numerical score only."
    evaluations['coherence'] = create_evaluation_query(coherence_template, prompt=prompt, generated_text=generated_text)

    # Evaluate Detail and Exhaustiveness
    detail_template = "Assessing the detail and exhaustiveness relative to the prompt '{prompt}', how thoroughly does this generated text: '{generated_text}' cover the topic? Rate on a scale from 0 to 1, where 1 is very detailed and exhaustive, and 0 is not detailed at all. Provide a numerical score only."
    evaluations['details'] = create_evaluation_query(detail_template, prompt=prompt, generated_text=generated_text)

    # Evaluate Suitability as an Answer
    suitability_template = "Evaluate the suitability of this generated text: '{generated_text}' as an answer to the original prompt '{prompt}'. On a scale from 0 to 1, where 1 is a perfect answer and 0 is completely unsuitable, provide a numerical score only."
    evaluations['suitability'] = create_evaluation_query(suitability_template, prompt=prompt, generated_text=generated_text)

    return evaluations

def critique(model, prompt, generated_text):
    evaluation_weights = {
        'relevance': 3,  
        'clarity': 1,
        'coherence': 0.5,
        'details': 1.5,
        'suitability': 2  
    }
    
    evaluations = evaluate_with_llm(model, prompt, generated_text)
    print("Evaluations:", evaluations)
    
    # Calculate the weighted sum of the evaluations
    weighted_sum = sum(evaluations[aspect] * evaluation_weights.get(aspect, 1) for aspect in evaluations)
    
    # Calculate the sum of weights for the aspects evaluated
    total_weight = sum(evaluation_weights.get(aspect, 1) for aspect in evaluations)
    
    # Calculate the weighted average of the evaluations
    weighted_average = weighted_sum / total_weight if total_weight > 0 else 0
    
    return [weighted_average, evaluations]

In [29]:
evaluationLLm =  ChatGoogleGenerativeAI(model="gemini-1.0-pro-latest")

In [26]:
def generate_queries(model,prompt, num_queries):
  query_generation_prompt = ChatPromptTemplate.from_template("Given the prompt: '{prompt}', generate {num_queries} questions that are better articulated. Return in the form of an list. For example: ['question 1', 'question 2', 'question 3']")
  query_generation_chain = query_generation_prompt | model
  return query_generation_chain.invoke({"prompt": prompt, "num_queries": num_queries}).content.split('\n')

In [33]:
question = 'My patient has had a full ACL reconstruction and repair following an ACL tear and a partial meniscus tear. To help him recover fully, what steps are required? How should we prevent a retear of the ACL ligament?'

In [34]:
queries = generate_queries(evaluationLLm,question,3)
retrievedDocs = cohereRetrival("MedicalPapers",queries[:1])

In [35]:
queries

['1. What are the essential stages of rehabilitation to optimize recovery after ACL reconstruction and meniscus repair?',
 '2. How can we effectively prevent a retear of the ACL ligament during the rehabilitation process?',
 '3. What are the specific precautions and adaptations required to minimize the risk of retear during functional activities post-rehabilitation?']

In [48]:
len(retrievedDocs)

10

In [55]:
retrievedDocs[0].payload.values()

dict_values(['1 \n University Orthopedics, Inc.  \nSports Medicine Division  \n \nANTERIOR  CRUCIATE  LIGAMENT  (ACL ) WITH MENISCUS REPAIR  POST -OP \nREHABILITATION  PROTOCOL  \n \nThe following is a protocol for postoperative patients following ACL reconstruction  and meniscus repair  (medial \nor lateral) .  The primary goal of this protocol is to protect t he reconstruction and while steadily progressing \ntowards and ultimately achieving pre -injury level of activity.  Please note this protocol is a guideline .  Patients \nwith additional surgery (i.e. collateral ligament repair ) will progress at dif ferent rates.  Achieving the criteria of \neach phase should be emphasized more than the approximate duration.  If a patient should develop an increase \nin pain or swelling or decrease in motion at any time, act ivity should be decreased until problems are res olved.  \n \nPHASE I :  0-2 WEEKS POSTOPERATIVE  \n \nGOALS:  \n• Progressive decrease in swelling  \n• AROM:  0 – 90 degre

In [57]:
for doc in retrievedDocs:
    for key,value in doc.payload.items():
        print(key,value)

SyntaxError: expected ':' (4028238099.py, line 2)

In [43]:
def cohereRetrival(collection,textList):
    cohere_client = cohere.Client(os.environ["COHERE_API_KEY"])
    client = qdrant_client.QdrantClient()
    result = client.search(
    collection_name=collection,
    query_vector=cohere_client.embed(
        model="embed-english-v3.0",  
        input_type="search_query",  # Input type for search queries
        texts=textList,
    ).embeddings[0],
    )
    return result

In [42]:
def getResponce(prompt,context = "",PatientHistory=""):
    result = chain.invoke({"context":context,"patientHistory":PatientHistory,"question":prompt})
    return result

In [ ]:
def ragFusion(prompt,collection= "MedicalPapers"):
    queryGenerationPrompt = ChatPromptTemplate.from_template("Given the prompt: '{prompt}', generate {num_queries} questions that are better articulated. Return in the form of an list. For example: ['question 1', 'question 2', 'question 3']")
    queryGenerationChain = queryGenerationPrompt | model
    queries = query_generation_chain.invoke({"prompt": prompt, "num_queries": num_queries}).content.split('\n')
    retrievedContent = []
    for query in queries:
        ret = cohereRetrival(collection,query)
        